NameError: name 'nvidia' is not defined

In [ ]:
import torch
print(torch.cuda.is_available())       # Should return True
print(torch.cuda.device_count())       # Should return number of GPUs
print(torch.cuda.get_device_name(0))   # Should return your GPU name


In [15]:
!{sys.executable} -m pip install torch==2.8.0+cu126 torchvision==0.23.0+cu126 torchaudio==2.8.0+cu126 --index-url https://download.pytorch.org/whl/cu126
#!{sys.executable} -m pip install --user paddleocr
#!{sys.executable} -m pip install --user "paddlepaddle==2.6.1" -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 7.6 MB/s eta 0:06:23
   ---------------------------------------- 0.0/2.9 GB 8.7 MB/s eta 0:05:35
   ---------------------------------------- 0.0/2.9 GB 9.7 MB/s eta 0:05:00
   ---------------------------------------- 0.0/2.9 GB 9.1 MB/s eta 0:05:21
   ---------------------------------------- 0.0/2.9 GB 9.4 MB/s eta 0:05:12
   ---------------------------------------- 0.0/2.9 GB 9.9 MB/s eta 0:04:56
   ---------------------------------------- 0.0/2.9 GB 9.6 MB/s eta 0:05:03
   ---------------------------------------- 0.0/2.9 GB 8.2 MB/s eta 0:05:58
   ---------------------------------------- 0.0/2.9 GB 8.3 MB/s eta 0:05:53
   ---------------------------------------- 0.0/2.9 GB 8.8 MB/s eta 0:05:31
   ---------------------------------------- 0.0/2.9 GB 9.1 MB/s eta 0:05:19
   ---------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#!{sys.executable} -m pip install Pillow

from pathlib import Path
from PIL import Image
import easyocr
import numpy as np
from sklearn.cluster import DBSCAN
import numpy as np
import os
from pathlib import Path
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont



c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import easyocr
import re

# --------------------------
# CONFIG
# --------------------------
input_folder = Path(r"input")
output_folder = Path("output")
output_folder.mkdir(exist_ok=True)

allow_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?.,-…'\""
replacements = {"PottyWords": "&$%&#$%"}
font_path = "fonts/CCMeanwhileBoldItalic.ttf"
font_size = 22

# --------------------------
# INITIALIZE OCR
# --------------------------
reader = easyocr.Reader(['en'], gpu=True)

# --------------------------
# HELPER FUNCTIONS
# --------------------------
def natural_sort_key(path):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'(\d+)', str(path))]

def replace_words(results, replacements):
    new_results = []
    for bbox, text, conf in results:
        new_text = " ".join([replacements.get(word.upper(), word) for word in text.split()])
        new_results.append((bbox, new_text, conf))
    return new_results

def draw_text_on_image_no_overlap(img, results, font_path, font_size, padding=25):
    pil_img = img.copy()
    draw = ImageDraw.Draw(pil_img)
    font = ImageFont.truetype(font_path, font_size)

    # Step 1: Draw all white backgrounds
    for bbox, text, conf in results:
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        bbox_text = draw.textbbox((0, 0), text, font=font)
        text_width = bbox_text[2] - bbox_text[0]
        text_height = bbox_text[3] - bbox_text[1]
        x0 = x_center - text_width // 2 - padding
        y0 = y_center - text_height // 2 - padding
        x1 = x_center + text_width // 2 + padding
        y1 = y_center + text_height // 2 + padding
        draw.rectangle([x0, y0, x1, y1], fill="white")

    # Step 2: Draw text
    for bbox, text, conf in results:
        x_center = int(np.mean([p[0] for p in bbox]))
        y_center = int(np.mean([p[1] for p in bbox]))
        draw.text((x_center, y_center), text, fill="black", font=font, anchor="mm")

    return pil_img

# --------------------------
# PROCESS FOLDERS
# --------------------------
# Get all folders (including input_folder itself)
folders = [f for f in input_folder.rglob('*') if f.is_dir()] + [input_folder]

for folder in sorted(set(folders)):
    # Gather images (jpg, jpeg, webp)
    image_files = sorted(
        list(folder.glob("*.webp")) +
        list(folder.glob("*.jpg")) +
        list(folder.glob("*.jpeg")),
        key=natural_sort_key
    )

    if not image_files:
        continue  # skip empty folders

    print(f"\nProcessing folder: {folder}")

    # OCR + replacement
    ocr_results_all = []
    for img_path in image_files:
        img = Image.open(img_path)
        results = reader.readtext(np.array(img), allowlist=allow_chars, width_ths=0.05)
        ocr_results_all.append(replace_words(results, replacements))

    # Output subfolder mirrors input structure
    relative_path = folder.relative_to(input_folder)
    output_subfolder = output_folder / relative_path
    output_subfolder.mkdir(parents=True, exist_ok=True)

    # Save processed images
    for idx, img_path in enumerate(image_files):
        img = Image.open(img_path)
        modified_img = draw_text_on_image_no_overlap(img, ocr_results_all[idx], font_path, font_size)
        output_path = output_subfolder / f"{img_path.stem}.png"
        modified_img.save(output_path)
        print(f"  Saved: {output_path}")


In [12]:
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.current_device())
    print(torch.cuda.get_device_name(torch.cuda.current_device()))
    print(torch.version.cuda)


False


In [ ]:
nvcc --version